In [4]:
import pandas as pd

In [5]:
data = pd.read_excel('Base_de_datos_de_Comprasal-proporcionado_por_MH.xls')

In [6]:
data.columns

Index(['Institucion', 'Tipo de Proceso', 'Codigo Proceso', 'Nombre de Proceso',
       'Resultado', 'Monto', 'Nombre Proveedor', 'Fecha Resultado'],
      dtype='object')

In [7]:
data.shape

(65535, 8)

In [8]:
data.head()

,Institucion,Tipo de Proceso,Codigo Proceso,Nombre de Proceso,Resultado,Monto,Nombre Proveedor,Fecha Resultado
0,Academia Nacional de Seguridad Pública,Contratación Directa (CD),CD-01/2019-ANSP,SERVICIO DE MANTENIMIENTO PREVENTIVO Y CORRECT...,Adjudicada,6000.00,"E- BUSINESS DISTRIBUTION DE EL SALVADOR, S.A. ...",2019-02-13 00:00:00
1,Academia Nacional de Seguridad Pública,Contratación Directa (CD),CD-02/2019-ANSP,MANTENIMIENTO PREVENTIVO CORRECTIVO DE PLANTA ...,Adjudicada,5766.34,"FASOR, S. A. DE C. V.",2019-01-31 00:00:00
2,Academia Nacional de Seguridad Pública,Contratación Directa (CD),CD-03/2018-ANSP,"SERVICIO DE SOPORTE PARA SOFTWARE ORACLE, DURA...",Adjudicada,13953.95,"DATUM, S.A. DE C.V.",2018-09-19 00:00:00
3,Academia Nacional de Seguridad Pública,Libre Gestion,LG-0159-2018,SUMINISTRO DE CARTONES PARA FIJAR SILUETAS,Oferta Seleccionada,3060.00,"IMPRENTA LA TARJETA, S. A. DE C. V.",2018-04-23 00:00:00
4,Academia Nacional de Seguridad Pública,Libre Gestion,LG-0160-2018,SUMINISTRO DE PLACAS Y MEDALLAS PARA GRADUACIO...,Cierre del Proceso sin Seleccion,0.00,NaN,2018-04-04 00:00:00


In [9]:
data.Resultado.unique()

array(['Adjudicada', 'Oferta Seleccionada',
       'Cierre del Proceso sin Seleccion', 'Sin Resultado', 'Formalizado',
       'Desierta', 'Sin Efecto', 'Adjudicado',
       datetime.datetime(2018, 12, 17, 0, 0)], dtype=object)

In [10]:
data = data[data.Monto != 0]
data.shape

(47630, 8)

In [11]:
data.Resultado.unique()

array(['Adjudicada', 'Oferta Seleccionada', 'Formalizado',
       'Sin Resultado', 'Cierre del Proceso sin Seleccion', 'Adjudicado',
       datetime.datetime(2018, 12, 17, 0, 0)], dtype=object)

In [11]:
data[data.Resultado == 'Cierre del Proceso sin Seleccion']

,Institucion,Tipo de Proceso,Codigo Proceso,Nombre de Proceso,Resultado,Monto,Nombre Proveedor,Fecha Resultado
3143,Banco de Fomento Agropecuario (BFA),Libre Gestion,20180044,ADQUISICION DE 107 TEMPORIZADORES PROGRAMABLES...,Cierre del Proceso sin Seleccion,5243.00,"DISTRIBUIDORA GRANADA, S. A. DE C. V.",2018-06-04 00:00:00
5944,Centro Nacional de Tecnología Agropecuaria y F...,Libre Gestion,20180100,"ADQUISICIÓN DE 1,000 ENVASES PLÁSTICOS DE 750 ...",Cierre del Proceso sin Seleccion,400.00,JOSE EDGARDO HERNANDEZ PINEDA,2018-09-03 00:00:00
32049,Instituto Salvadoreño de Formación Profesional...,Libre Gestion,20180517,RECARGA DE 66 EXTINTORES PROPIEDAD DE INSAFORP...,Cierre del Proceso sin Seleccion,981.29,CENTRO INTEGRAL DE ASESORIA Y DESARROLLO EMPRE...,2018-05-31 00:00:00
32703,Instituto Salvadoreño de Formación Profesional...,Libre Gestion,20181170,PUBLICACIÓN DE AVISO DE RESULTADO DE LA LICITA...,Cierre del Proceso sin Seleccion,244.08,EDITORIAL ALTAMIRANO MADRIZ S.A.,2018-05-07 00:00:00
48702,Municipalidad de Chalchuapa,Libre Gestion,20180128,1 KIT DE RODILLOS DE CARAGA PAPEL EPSON L220,Cierre del Proceso sin Seleccion,121.00,ROBERTO CARLOS GARC (DIGITAL SOLUTIONS),2018-02-19 00:00:00
52247,Municipalidad de Izalco,Libre Gestion,20180511,compra de 1 deposito de agua de radiador para ...,Cierre del Proceso sin Seleccion,282.50,"CANJURA, S.A. DE C.V.",2018-05-11 00:00:00
58052,Municipalidad de San Salvador,Libre Gestion,20181512,"COMPRA DE 190 CAPAS,2 SOMBRILLAS Y 4 PARAGUAS",Cierre del Proceso sin Seleccion,4215.96,ARMANDO JOHNSON OVANDO,2018-07-30 00:00:00
60110,Municipalidad de Santa Cruz Michapa,Libre Gestion,20180004,Adquisición de 3000 quintales de abono.,Cierre del Proceso sin Seleccion,40500.00,"TURCIOS , JOSE ALONSO",2018-06-08 00:00:00


In [21]:
proveedores = data[['Nombre Proveedor', 'Monto']].groupby(['Nombre Proveedor']).sum().sort_values('Monto', ascending=False) / 1e6

In [22]:
proveedores.to_excel('proveedores.xlsx')

In [29]:
data[data['Nombre Proveedor'] == 'GOLD WILL, S.A. DE C.V.']

,Institucion,Tipo de Proceso,Codigo Proceso,Nombre de Proceso,Resultado,Monto,Nombre Proveedor,Fecha Resultado
593,Administración Nacional de Acueductos y Alcant...,Licitación Pública (LP),LP 19/2018,¿SUMINISTRO DE POLICLORURO DE ALUMINIO (PAC) P...,Adjudicada,1154345.85,"GOLD WILL, S.A. DE C.V.",2018-06-13 00:00:00
607,Administración Nacional de Acueductos y Alcant...,Licitación Pública (LP),LP-08/2018,"SUMINISTRO 411,000 KGS DE HIPOCLORITO DE CALCI...",Adjudicada,1044967.50,"GOLD WILL, S.A. DE C.V.",2018-05-08 00:00:00
786,Administración Nacional de Acueductos y Alcant...,Libre Gestion,20180116,"SUMINISTRO DE SULFATO FÉRRICO, PARA EL TRATAMI...",Sin Resultado,70003.50,"GOLD WILL, S.A. DE C.V.",2018-07-30 00:00:00
823,Administración Nacional de Acueductos y Alcant...,Libre Gestion,20180140,LIBRE GESTION Nº144/2018DENOMINADO SUMINISTRO...,Oferta Seleccionada,395.50,"GOLD WILL, S.A. DE C.V.",2018-10-05 00:00:00
1104,Administración Nacional de Acueductos y Alcant...,Libre Gestion,20180378,"LIBRE GESTION Nº17/2019, DENOMINADO SUMINISTR...",Oferta Seleccionada,66670.00,"GOLD WILL, S.A. DE C.V.",2019-01-02 00:00:00
8448,Comisión Ejecutiva Portuaria Autónoma (CEPA),Licitación Abierta (DR-CAFTA) (ADACA-UE),DR-CAFTA-ADACA-UE CEPA-LA-19/2018,SUMINISTRO DE DOS (2) GRÚAS MÓVILES POST PANAM...,Adjudicada,8904000.00,"GOLD WILL, S.A. DE C.V.",2018-06-29 00:00:00
39822,Ministerio de Gobernación (MIGOB),Libre Gestion,20180180,ADQUISICIÓN DE VEHÍCULOS TIPO PANEL PARA LA DI...,Oferta Seleccionada,28306.50,"GOLD WILL, S.A. DE C.V.",2018-10-25 00:00:00
61998,Municipalidad de Tecoluca,Libre Gestion,20180026,SUMINISTRO DE 1- VOLANTE PARA CAMIÓN RECOLECTO...,Oferta Seleccionada,141.25,"GOLD WILL, S.A. DE C.V.",2018-01-05 00:00:00
62004,Municipalidad de Tecoluca,Libre Gestion,20180032,"SUMINISTRO DE 1- JUEGO DE BARRA PARA PICK UP, ...",Oferta Seleccionada,169.50,"GOLD WILL, S.A. DE C.V.",2018-01-09 00:00:00
62036,Municipalidad de Tecoluca,Libre Gestion,20180059,"SUMINISTRO DE 1- BOMBA DE AGUA, 2- BALEROS Y 1...",Oferta Seleccionada,468.95,"GOLD WILL, S.A. DE C.V.",2018-02-14 00:00:00


In [13]:
anda = data[data['Institucion'].apply(lambda x: x.find('Administración Nacional de Acueductos') >= 0)]

In [16]:
anda

,Institucion,Tipo de Proceso,Codigo Proceso,Nombre de Proceso,Resultado,Monto,Nombre Proveedor,Fecha Resultado
562,Administración Nacional de Acueductos y Alcant...,Contratación Directa (CD),CD 10/2018,¿SUMINISTRO DE MATERIALES PARA COLUMNA DE SUCC...,Adjudicada,392197.47,SERVICIO AGRICOLA SALVADOREÑA,2018-10-15 00:00:00
563,Administración Nacional de Acueductos y Alcant...,Contratación Directa (CD),CD 10/2018,¿SUMINISTRO DE MATERIALES PARA COLUMNA DE SUCC...,Adjudicada,332620.22,"HIDRO SYSTEMS INC, SOCIEDAD ANONIMA DE CAPITAL...",2018-10-15 00:00:00
564,Administración Nacional de Acueductos y Alcant...,Contratación Directa (CD),CD-01/2018-URG,¿RECONSTRUCCIÓN Y MEJORAS EN LÍNEA DE IMPELENC...,Adjudicada,223000.00,"MONELCA,S.A. DE C.V.",2018-07-04 00:00:00
566,Administración Nacional de Acueductos y Alcant...,Contratación Directa (CD),CD-02/2018-URG,¿ARRENDAMIENTO DE CAMIONES CISTERNAS POR EMERG...,Adjudicada,6539.31,BALTAZAR DE JESUS BONILLA BONILLA,2018-07-12 00:00:00
567,Administración Nacional de Acueductos y Alcant...,Contratación Directa (CD),CD-02/2018-URG,¿ARRENDAMIENTO DE CAMIONES CISTERNAS POR EMERG...,Adjudicada,13560.00,"MARTINEZ ESCOBAR, S.A. DE C.V.",2018-07-12 00:00:00
568,Administración Nacional de Acueductos y Alcant...,Contratación Directa (CD),CD-02/2018-URG,¿ARRENDAMIENTO DE CAMIONES CISTERNAS POR EMERG...,Adjudicada,15000.07,"MENDEZ , SALVADOR",2018-07-12 00:00:00
569,Administración Nacional de Acueductos y Alcant...,Contratación Directa (CD),CD-02/2018-URG,¿ARRENDAMIENTO DE CAMIONES CISTERNAS POR EMERG...,Adjudicada,7500.04,"GLOBAL PARTS, S.A. DE C.V.",2018-07-12 00:00:00
570,Administración Nacional de Acueductos y Alcant...,Contratación Directa (CD),CD-03/2018,PERFORACION DE POZO Y SUMINISTRO E INSTALACION...,Adjudicada,317293.48,"A.P. & G, CONSTRUCTORES, SOCIEDAD ANONIMA DE C...",2018-06-21 00:00:00
571,Administración Nacional de Acueductos y Alcant...,Contratación Directa (CD),CD-03/2018,PERFORACION DE POZO Y SUMINISTRO E INSTALACION...,Adjudicada,325544.33,"CONSTRUCTORA VENTURA ZARRUK, S.A. DE C.V.",2018-06-21 00:00:00
572,Administración Nacional de Acueductos y Alcant...,Contratación Directa (CD),CD-04/2018,PERFORACIÓN DE POZO Y SUMINISTRO E INSTALACIÓN...,Adjudicada,218874.15,SERVICIO AGRICOLA SALVADOREÑA,2018-07-05 00:00:00


In [22]:
data['Nombre Proveedor'].dtype

dtype('O')

In [23]:
data[data['Nombre Proveedor'].apply(lambda x: str(x).lower().find('puntual') >= 0)]

,Institucion,Tipo de Proceso,Codigo Proceso,Nombre de Proceso,Resultado,Monto,Nombre Proveedor,Fecha Resultado
603,Administración Nacional de Acueductos y Alcant...,Licitación Pública (LP),LP-04/2019,CONTRATACION DE SERVICIOS DE RECUPERACION DE M...,Adjudicada,3500005.5,"PUNTUAL, S.A. DE C.V.",2019-02-28 00:00:00
